<center><span style="font-size:28px; font-weight:bold;">Implementez un modèle de scoring</span><center>

<center><span style="font-size:28px; font-weight:bold;">2 - Modélisation</span><center>

# 1 - Importation des librairies et des données

## 1.1 - Importation des différentes librairies

In [10]:
import mlflow
import mlflow.sklearn

# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Autres bibliothèques utiles :
import time

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [2]:
# On va démarrer le chronomètre :
start = time.time()

## 1.2 - Importation des données

In [ ]:
# Initialiser MLFlow
mlflow.set_experiment("Home Credit Default Risk - Modelisation")

In [ ]:
mlflow.start_run(run_name="Modélisation 1")

In [3]:
# Spécifions les différents chemins possibles
directories = [
    "C:\\Users\\mauge\\Documents\\github\\P7_implementer_modele_scoring",
    "/Users/Gary/Documents/GitHub/FormationData/DataScientist_P7/Projet+Mise+en+prod+-+home-credit-default-risk",
    "/Users/Gary/Documents/GitHub/P7_implementer_modele_scoring",
    "C:\\Users\\mauge\\OneDrive\\Bureau\\Projet+Mise+en+prod+-+home-credit-default-risk"
]

In [12]:
def load_csv(file_name, directory_options, **kwargs):
    for directory in directory_options:
        file_path = os.path.join(directory, file_name)
        if os.path.exists(file_path):
            print(f"Loading {file_name} from {file_path}")
            return pd.read_csv(file_path, **kwargs)
    print(f"{file_name} not found in any of the specified directories.")
    return None

# Chargement des fichiers
#df_application_train = load_csv('df_final.csv', directories)
X_train = load_csv('X_train.csv', directories)
y_train = load_csv('y_train.csv', directories)
X_test = load_csv('X_test.csv', directories)
y_test = load_csv('y_test.csv', directories)

Loading X_train.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/X_train.csv
Loading y_train.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/y_train.csv
Loading X_test.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/X_test.csv
Loading y_test.csv from /Users/Gary/Documents/GitHub/P7_implementer_modele_scoring/y_test.csv


In [5]:
X_train.head()

,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0.0,0.0,1.0,0.0,0.001512,0.090287,0.090032,0.077441,0.256702,0.687292,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.002089,0.311736,0.132924,0.271605,0.045506,0.832143,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,1.0,1.0,0.0,0.000358,0.022472,0.020025,0.023569,0.135340,0.877380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.000935,0.066837,0.109477,0.063973,0.107480,0.876567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.000819,0.116854,0.078975,0.117845,0.393191,0.894951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2 - Premier modèle de prédiction

## 2.1 - Prédiction par régression linéaire

Cette régression linéaire va nous permettre d'avoir une base.

In [13]:
# Définition des modèles :
models = {
    'Linear Regression': LinearRegression()
}

In [15]:
# Initialisation d'un DataFrame pour stocker les résultats des modèles
results_df = pd.DataFrame(columns=['Model', 'R2_mean', 'MAE_mean', 'RMSE_mean', 'Fit_time_mean', 'Score_time_mean'])

# Évaluation de chaque modèle avec cross_validate
for name, model in models.items():
    scores = cross_validate(model, X_train, y_train, cv=5, 
                            scoring={'r2': 'r2', 'MAE': 'neg_mean_absolute_error', 'RMSE': 'neg_root_mean_squared_error'},
                            return_train_score=False, n_jobs=-1)
    
    # On calcule la moyenne des scores et des temps
    new_row = pd.DataFrame({
        'Model': [name],
        'R2_mean': [np.mean(scores['test_r2'])],
        'MAE_mean': [-np.mean(scores['test_MAE'])],
        'RMSE_mean': [-np.mean(scores['test_RMSE'])],
        'Fit_time_mean': [np.mean(scores['fit_time'])],
        'Score_time_mean': [np.mean(scores['score_time'])]
    })
    
    # Utilisation de pd.concat pour ajouter la nouvelle ligne
    results_df = pd.concat([results_df, new_row], ignore_index=True)

# On affiche les résultats :
print(results_df)

               Model       R2_mean      MAE_mean     RMSE_mean  Fit_time_mean  \
0  Linear Regression -9.017217e+15  46489.594779  1.152929e+07       44.99408   

   Score_time_mean  
0         0.535304  


## 2.2 - Prédiction par

In [ ]:
elapsed_time = time.time() - start  # Calcule le temps écoulé
print(f"Temps total d'exécution du notebook : {elapsed_time:.2f} secondes.")